In [1]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.cudnn.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


In [2]:

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # 填充token设置

# 加载原始模型
base_model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# 加载LoRA微调后的模型
from peft import PeftModel
lora_model = PeftModel.from_pretrained(base_model, "./results/lora_finetuned_model").to(device)

prompt = "Once upon a time"

inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)


In [3]:

with torch.no_grad():
    # 原始模型生成
    base_output = base_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
    )
    base_text = tokenizer.decode(base_output[0], skip_special_tokens=True)

    # 重置随机种子，保证对比公平
    set_seed(42)

    # LoRA微调模型生成
    lora_output = lora_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
    )
    lora_text = tokenizer.decode(lora_output[0], skip_special_tokens=True)

print("=== Base Model Output ===")
print(base_text)

print("\n=== LoRA Fine-tuned Model Output ===")
print(lora_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Base Model Output ===
Once upon a time the universe became a mere place of chaos and disorder, the stars and planets became more and more massive and dense, and the oceans became increasingly contaminated by the pollution of the atmosphere. The world was made to resemble a vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast

=== LoRA Fine-tuned Model Output ===
Once upon a time the universe became a mere place of chaos and disorder, the stars and planets became more and more massive and dense, and the oceans became increasingly contaminated by the pollution of the atmosphere. The world was made to resemble a vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast, vast


In [ ]:
base_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D(nf=2304, nx=768)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=Fal

In [5]:
lora_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           